## Model Evaluation - Keras Embedding
### Balanced

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, roc_auc_score, RocCurveDisplay, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model
from scipy.sparse import load_npz
from pickle import load

import re

Load the test sets

In [ ]:
# logistic regression and neural network
test_feats_file = "~/scratch/datasets/yale_new_haven/training_test_sets/balanced_dataset/features/normalized_preprocessing/regression_nn/yale_new_haven_balanced_test_features.csv"
test_labels_file = "~/scratch/datasets/yale_new_haven/training_test_sets/balanced_dataset/labels/yale_new_haven_balanced_test_labels.csv"

In [ ]:
X_test = pd.read_csv(test_feats_file)
y_test = pd.read_csv(test_labels_file)

In [ ]:
test_ids = X_test['ID'].astype("int32")

In [ ]:
X_test = X_test[[col for col in X_test.columns if col != 'ID']]

In [ ]:
disposition_var = {'disposition'}
demographic_vars = {'age', 'gender', 'ethnicity', 'race', 'lang',
       'religion', 'maritalstatus', 'employstatus', 'insurance_status'}
triage_evaluation_vars = {'dep_name', 'esi', 'arrivalmode', 'arrivalmonth', 'arrivalday', 'arrivalhour_bin'}.union({col for col in X_test.columns if 'triage_vital' in col})
chief_complaint_vars = {col for col in X_test.columns if "cc_" in col}
medication_vars = {col for col in X_test.columns if 'meds_' in col}
hospital_usage_stats_vars = {'previousdispo', 'n_edvisits', 'n_admissions', 'n_surgeries'}
imaging_ekg_vars = {'cxr_count','echo_count','ekg_count','otherxr_count', 'otherus_count', 'headct_count', 'otherct_count', 'mri_count','otherimg_count'}
historical_vital_vars = {'dbp_last',
 'dbp_max',
 'dbp_median',
 'dbp_min',
 'o2_device_last',
 'o2_device_max',
 'o2_device_median',
 'o2_device_min',
 'pulse_last',
 'pulse_max',
 'pulse_median',
 'pulse_min',
 'resp_last',
 'resp_max',
 'resp_median',
 'resp_min',
 'sbp_last',
 'sbp_max',
 'sbp_median',
 'sbp_min',
 'spo2_last',
 'spo2_max',
 'spo2_median',
 'spo2_min',
 'temp_last',
 'temp_max',
 'temp_median',
 'temp_min'}
curr = disposition_var.union(demographic_vars.union(triage_evaluation_vars.union(chief_complaint_vars.union(medication_vars.union(hospital_usage_stats_vars.union(imaging_ekg_vars.union(historical_vital_vars)))))))
past_medical_hist_vars = {col for col in X_test.columns if col not in curr and "_" not in col and col not in ['ID', 'previousdispo']}

cc_cols = list(chief_complaint_vars)
pmh_cols = list(past_medical_hist_vars)

cc_cols.sort()
pmh_cols.sort()

In [ ]:
cc_sparse_features = load_npz('/home/mila/d/david.hobson/scratch/datasets/yale_new_haven/training_test_sets/balanced_dataset/features/keras_embeddings/cc_embedding_test_input.npz')
cc_features = pd.DataFrame(cc_sparse_features.todense())

In [ ]:
pmh_sparse_features = load_npz('/home/mila/d/david.hobson/scratch/datasets/yale_new_haven/training_test_sets/balanced_dataset/features/keras_embeddings/pmh_embedding_test_input.npz')
pmh_features = pd.DataFrame(pmh_sparse_features.todense())

In [ ]:
other_features = X_test[[col for col in X_test.columns if col not in cc_cols and col not in pmh_cols]]

In [ ]:
X_test = {'other': other_features, 'cc': cc_features, 'pmh': pmh_features}

Load the models

In [ ]:
nn_filepath = "/home/mila/d/david.hobson/scratch/models/balanced/experiments/keras_embedding"

In [ ]:
nn_model = load_model(nn_filepath)

### Test set probability

In [ ]:
nn_loss, nn_acc = nn_model.evaluate(X_test, y_test)

In [ ]:
print(f"Test Accuracy:\n\t\n\tNeural Network: {np.round(nn_acc*100, 2)}")

## AUROC

### Scores

In [ ]:
# Hong: AUROC = 0.91
nn_auroc = roc_auc_score(y_test, nn_model.predict(X_test))

In [ ]:
print(f"Test AUROC:\n\t\n\tNeural Network: {np.round(nn_auroc*100, 2)}")

### Plots

In [ ]:
RocCurveDisplay.from_predictions(y_test, nn_model.predict(X_test))
plt.show()

### Seniors vs. Adults

In [ ]:
senior_idxs_filename = "/home/mila/d/david.hobson/scratch/datasets/yale_new_haven/demographic_indices/age_65_and_over.pickle"
adult_idxs_filename = "/home/mila/d/david.hobson/scratch/datasets/yale_new_haven/demographic_indices/age_less_than_65.pickle"

In [ ]:
# indices of patients 65 and older
with open(senior_idxs_filename, 'rb') as f:
    senior_idxs = load(f)
    
with open(adult_idxs_filename, 'rb') as f:
    adult_idxs = load(f)

In [ ]:
seniors_test = test_ids[test_ids.isin(senior_idxs)].index
adults_test = test_ids[~test_ids.isin(senior_idxs)].index

In [ ]:
X_test_seniors = {'other': other_features.loc[seniors_test], 'cc': cc_features.loc[seniors_test], 'pmh': pmh_features.loc[seniors_test]}
X_test_adults = {'other': other_features.loc[adults_test], 'cc': cc_features.loc[adults_test], 'pmh': pmh_features.loc[adults_test]}

Seniors vs. Adults Scores

In [ ]:
nn_loss_seniors, nn_acc_seniors = nn_model.evaluate(X_test_seniors, y_test.loc[seniors_test])
nn_loss_adults, nn_acc_adults = nn_model.evaluate(X_test_adults, y_test.loc[adults_test])

In [ ]:
# print(f"Training set:\n\tAdults score: {np.round(adult_score_train*100, 2)}\n\tSeniors score: {np.round(senior_score_train*100, 2)}")
print("Test Accuracy")
print()
print(f"Neural Network:\n\tAdults score: {np.round(nn_acc_adults*100, 2)} (n = {len(adults_test)})\n\tSeniors score: {np.round(nn_acc_seniors*100, 2)} (n = {len(seniors_test)})")

### Confusion Matrix

In [ ]:
y_pred_nn = np.round(nn_model.predict(X_test))

y_pred_nn_seniors = np.round(nn_model.predict(X_test_seniors))
y_pred_nn_adults = np.round(nn_model.predict(X_test_adults))

Overall

In [ ]:
print("Overall")
print('\tNeural Network')
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_nn, normalize='all')

In [ ]:
print("Overall")
print('\tNeural Network')
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_nn, normalize='all')

Seniors

In [ ]:
print('Seniors')
print('\tNeural Network')
ConfusionMatrixDisplay.from_predictions(y_test.loc[seniors_test], y_pred_nn_seniors, normalize='true')

Adults

In [ ]:
print("Adults")
print('\tNeural Network')
ConfusionMatrixDisplay.from_predictions(y_test.loc[adults_test], y_pred_nn_adults, normalize='true')